# Instructions

1.  Download this dataset:  https://www.kaggle.com/slothkong/10-monkey-species  
        You may need to create a kaggle account / sign in to accomplish this.
        
2. Git clone this repo or download this notebook.
3. Create a new folder, data in the same directory this notebook is located
4. Unzip the monkey dataset in this folder.  Then unzip the train.zip and validation.zip folders.
5. The directory hierarchy should be ../data/10-monkey-species/training and ../data/10-monkey-species/validation.  Each of these subdirectories should contain a directory per label, n0 thru n9.

6. Implement the code in build_model()

** You will most certainly need access to a GPU for this assignment **

In [1]:
# these seeds are both required for reproducibility
import numpy as np
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
import os

Using TensorFlow backend.


In [2]:
class Configuration:
    def __init__(self):
        self.feature_extraction_epochs = 10
        self.fine_tuning_epochs = 20
        self.epochs_without_transfer_learning = 100
        self.batch_size = 30
        self.data_dir = "../input/10_monkey_species/10-monkey-species/training"
        self.val_dir = "../input/10_monkey_species/10-monkey-species/validation"

In [3]:
def build_model_feature_extraction():
    # create the base pre-trained model
    base_model = InceptionV3(weights='imagenet', include_top=False)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)  # make sure we're back at a 2d tensor
    x = Dense(1024, activation='relu')(x)  # add one fully connected layer
    predictions = Dense(10, activation='softmax')(x)  # 10 monkey, therefore 10 output units w softmax activation

    model = Model(inputs=base_model.input, outputs=predictions)

    # make the base model untrainable (frozen)
    for layer in base_model.layers:
        layer.trainable = False

    # compile the model (should be done *after* setting layers to non-trainable)
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    return model




In [4]:
def build_model_fine_tuning(model, learning_rate=0.0001, momentum=0.9):

        for layer in model.layers[:249]:
            layer.trainable = False
        for layer in model.layers[249:]:
            layer.trainable = True
        model.compile(optimizer=SGD(lr=learning_rate, momentum=momentum), loss='categorical_crossentropy', metrics=['accuracy'])
        return model

In [5]:
def create_callbacks(name):
    tensorboard_callback = TensorBoard(log_dir=os.path.join(os.getcwd(), "tensorboard_log", name), write_graph=True, write_grads=False)
    checkpoint_callback = ModelCheckpoint(filepath="./model-weights-" + name + ".{epoch:02d}-{val_loss:.6f}.hdf5", monitor='val_loss',
                                          verbose=0, save_best_only=True)
    return [tensorboard_callback]

In [6]:
def setup_data(train_data_dir, val_data_dir, img_width=299, img_height=299, batch_size=16):
    train_datagen = ImageDataGenerator(rescale=1./255)
    val_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical')

    validation_generator = val_datagen.flow_from_directory(
        val_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical')
    return train_generator, validation_generator

In [7]:
def fit_model(model, train_generator, val_generator, batch_size, epochs, name):
    model.fit_generator(
        train_generator,
        steps_per_epoch=train_generator.n // batch_size,
        epochs=epochs,
        validation_data=val_generator,
        validation_steps=val_generator.n // batch_size,
        callbacks=create_callbacks(name=name),
        verbose=1)
    return model

In [8]:
def eval_model(model, val_generator, batch_size):
    scores = model.evaluate_generator(val_generator, steps=val_generator.n // batch_size)
    print("Loss: " + str(scores[0]) + " Accuracy: " + str(scores[1]))

## Load configuration class
This has all our model configuration parameters in it.  It's defined above.

In [9]:
config = Configuration()

## Load Data
If everything is setup correctly, this method will print  
"  
Found 1097 images belonging to 10 classes.  
Found 272 images belonging to 10 classes.  
"  



In [10]:
train_generator, val_generator = setup_data(config.data_dir, config.val_dir, batch_size=config.batch_size)

Found 1097 images belonging to 10 classes.
Found 272 images belonging to 10 classes.


## Feature Extraction Model Training

Here we will train the feature extraction model

In [11]:
model = build_model_feature_extraction()
print (model.summary())

87916544/87910968 [==============================] - 10s 0us/step
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
___________________________________________

Homework Q1: How many trainable parameters does this model have? How many more parameters exist in this model than the model we built last week?

The model has 2,108,426 Trainable parameters, which is less than the original model that had 4,306,042 trainable parameters. Half the amount of trainable parameters. However this model has 23,911,210 total parameters, which is much more than the total of 4,306,042 parameters total in the last model.


Homework Q2: How can we possibly train a network of this size on a small dataset without overfitting?



In [12]:
model = fit_model(model, train_generator, val_generator,
                  batch_size=config.batch_size,
                  epochs=config.feature_extraction_epochs,
                  name="feature_extraction")

Epoch 1/10
36/36 [==============================] - 61s 2s/step - loss: 2.9308 - acc: 0.5098 - val_loss: 0.4678 - val_acc: 0.8529
Epoch 2/10
36/36 [==============================] - 57s 2s/step - loss: 0.5398 - acc: 0.8425 - val_loss: 0.3069 - val_acc: 0.9191
Epoch 3/10
36/36 [==============================] - 53s 1s/step - loss: 0.4226 - acc: 0.8731 - val_loss: 0.5867 - val_acc: 0.8934
Epoch 4/10
36/36 [==============================] - 56s 2s/step - loss: 0.2965 - acc: 0.9141 - val_loss: 0.2886 - val_acc: 0.8971
Epoch 5/10
36/36 [==============================] - 57s 2s/step - loss: 0.4532 - acc: 0.9051 - val_loss: 0.3690 - val_acc: 0.8860
Epoch 6/10
36/36 [==============================] - 55s 2s/step - loss: 0.1691 - acc: 0.9481 - val_loss: 0.2475 - val_acc: 0.9338
Epoch 7/10
36/36 [==============================] - 49s 1s/step - loss: 0.1535 - acc: 0.9630 - val_loss: 0.2047 - val_acc: 0.9485
Epoch 8/10
36/36 [==============================] - 60s 2s/step - loss: 0.1515 - acc: 0.95

In [13]:
eval_model(model, val_generator, batch_size=config.batch_size)

Loss: 0.31162813388639027 Accuracy: 0.9037036961979337


## Fine Tuning 

Here we will fine tune that same model

In [14]:
model = build_model_fine_tuning(model)
model = fit_model(model, train_generator, val_generator,
                      batch_size=config.batch_size,
                      epochs=config.fine_tuning_epochs,
                      name="fine_tuning")

Epoch 1/20
36/36 [==============================] - 61s 2s/step - loss: 0.0446 - acc: 0.9852 - val_loss: 0.0995 - val_acc: 0.9779
Epoch 2/20
36/36 [==============================] - 57s 2s/step - loss: 0.0370 - acc: 0.9880 - val_loss: 0.0884 - val_acc: 0.9816
Epoch 3/20
36/36 [==============================] - 56s 2s/step - loss: 0.0129 - acc: 0.9981 - val_loss: 0.0827 - val_acc: 0.9853
Epoch 4/20
36/36 [==============================] - 54s 1s/step - loss: 0.0143 - acc: 0.9981 - val_loss: 0.0799 - val_acc: 0.9890
Epoch 5/20
36/36 [==============================] - 57s 2s/step - loss: 0.0152 - acc: 0.9975 - val_loss: 0.0777 - val_acc: 0.9890
Epoch 6/20
36/36 [==============================] - 56s 2s/step - loss: 0.0099 - acc: 0.9972 - val_loss: 0.0784 - val_acc: 0.9890
Epoch 7/20
36/36 [==============================] - 55s 2s/step - loss: 0.0148 - acc: 0.9944 - val_loss: 0.0764 - val_acc: 0.9890
Epoch 8/20
36/36 [==============================] - 53s 1s/step - loss: 0.0078 - acc: 0.99

Homework Q3: Did fine tuning help significantly?

Yes, the base accuracy and loss were Loss: 0.31162813388639027 Accuracy: 0.9037036961979337 when looking at the validation set. After fine tuning we obtained the values of Loss: 0.0722703309988396 Accuracy: 0.9888888796170553 which results in a loss of 23% of the prior.

Homework Q4: Imagine we wanted to use the InceptionV3 model to create a classifier for detecting skin cancer.  You have 200 images of both cancerous and benign abnormal skin cells.  Do you expect Fine Tuning to be useful in solving this problem, or will feature selection be sufficient?

As we have seen, 200 images is a bit light in the data department. It would seem likely that there would be a high amount of variance due to the model being unable to properly generalize using such a small data set. We can use data augmentation to a degree, but ultimately fine tuning would likely be necessary. The problem is the fine tuning with inception v3 may not do well because in order to use it the domain of the classification materials should be the same or quite similar to that which you are training. If it doenst have similar features, then it will not work out, and it would be better to train from scratch. 

In [15]:
eval_model(model, val_generator, batch_size=config.batch_size)

Loss: 0.0722703309988396 Accuracy: 0.9888888796170553


In [ ]:
print("Saving Model...")
model.save("transfer_learning_model.h5")


Saving Model...
